# Running QiskitPattern using decorators (Experimental)

In this tutorial we will describe alternative way (interface) of running your patterns.

This new interface provides a way to define pattern as python function and run it in a single file, using `@distribute_qiskit_pattern` decorator.
All you need to do is annotate function with `@distribute_qiskit_pattern` decorator and call it. 
As a result of the call of the function you will get `Job` handle to check it's progress like we did in previous tutorials.

Limitations of this interface:

- Functions decorated with distribute_qiskit_pattern, can only accept named arguments for now. E.g do not use `my_pattern(argument1)`, instead specify name of the argument `my_pattern(argument1=argument1)`
- Function return will run `quantum_serverless.save_result` function under the hood, which means return values must be json serializable values in form of dictionary (with values as all Python native types, like strings, lists, dicts, `numpy` arrays, `QuantumCircuit`, `Operator`, etc.)
- When using local folder/modules user must specify `working_dir` as `./` (current folder), which will be archiving and sending content of entire folder for remote execution. Make sure that folder does not have large files. 

> &#x26A0; This interface is experimental, therefore it is subjected to breaking changes.

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit-extensions.github.io/quantum-serverless/deployment/local.html).

In [ ]:
import os
from quantum_serverless import ServerlessProvider

provider = ServerlessProvider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)
provider

<ServerlessProvider: gateway-provider>

## Hello, Qiskit!

Let's create simpliest pattern by writing a funtion `hello_qiskit` and annotating it with `@distribute_qiskit_pattern` decorator. 
The ``distribute_qiskit_pattern`` decorator accepts a [BaseProvider](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.BaseProvider.html) instance for the ``provider`` argument. Other arguments are `dependencies` to specify extra packages to install during execution and `working_dir` to specify working directory that will be shiped for remote execution if needed.

In [2]:
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

from quantum_serverless import distribute_qiskit_pattern, distribute_task, get, save_result


@distribute_qiskit_pattern(provider)
def hello_qiskit():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.measure_all()
    circuit.draw()

    sampler = Sampler()
    quasi_dists = sampler.run(circuit).result().quasi_dists

    return quasi_dists


job = hello_qiskit()
job

<Job | 96ad5b6a-f514-48fa-8ca2-927cc14e2c47>

In [3]:
job.result()

[{'0': 0.4999999999999999, '3': 0.4999999999999999}]

## QiskitPattern with distributed tasks

As in previous examples you can define distributed tasks and call them within a pattern.

In [4]:
from quantum_serverless import get_arguments, save_result, distribute_task, get
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit.circuit.random import random_circuit


@distribute_task(target={"cpu": 2})
def distributed_sample(circuit: QuantumCircuit):
    """Distributed task that returns quasi distribution for given circuit."""
    return Sampler().run(circuit).result().quasi_dists


@distribute_qiskit_pattern(provider)
def pattern_with_distributed_tasks(circuits):
    sample_task_references = [distributed_sample(circuit) for circuit in circuits]
    results = get(sample_task_references)
    print(results)


circuits = []
for _ in range(3):
    circuit = random_circuit(2, 2)
    circuit.measure_all()
    circuits.append(circuit)

job = pattern_with_distributed_tasks(circuits=circuits)
job

<Job | 6dbd75bc-bf32-4847-8621-bb0d247485cd>

In [5]:
job.result()
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
2023-10-27 12:46:57,031	INFO worker.py:1329 -- Using address 10.42.0.245:6379 set in the environment variable RAY_ADDRESS
2023-10-27 12:46:57,032	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 10.42.0.245:6379...
2023-10-27 12:46:57,063	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 10.42.0.245:8265 
(pid=119, ip=10.42.0.246) OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
[[{0: 1.0}], [{0: 1.0}], [{0: 0.065352163370718, 1: 0.434647836629282, 2: 0.065352163370718, 3: 0.434647836629282}]]
(pid=119, ip=10.42.0.246) OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs

## QiskitPattern with local modules/folders

Situation with local folders is a little bit trickier. In order to make local imports work in remote execution of a pattern we need to specify `working_dir` argument for `distribute_qiskit_pattern` decorator. It will tell quantum serverless to ship all content of current folder to remote cluster, which will make local folders discoverable by Python interpreter during remote execution.

In this example we will use local folder `source_files` with `circuit_utils.py` file, which has implementation of `create_hello_world_circuit` function. 

In [ ]:
import os
from quantum_serverless import distribute_qiskit_pattern, distribute_task, get, save_result
from qiskit.primitives import Sampler

from source_files.circuit_utils import create_hello_world_circuit


@distribute_qiskit_pattern(provider, working_dir="./")
def my_pattern_with_modules():
    quasi_dists = Sampler().run(create_hello_world_circuit()).result().quasi_dists
    return {"quasi_dists": quasi_dists}


job = my_pattern_with_modules()
job

In [ ]:
job.result()